In [1]:
import numpy as np
import torch
import torch_geometric as pyg
import graphPINN
import math
import logging
from time import time
from tqdm.notebook import tqdm
from scipy.io import savemat, loadmat
import os

In [2]:
folder = "/home/nasa/code/checkpoints/2023-04-03/"
if not os.path.exists(folder):
    os.makedirs(folder)

In [3]:
def pretty_size(n,pow=0,b=1024,u='B',pre=['']+[p+'i'for p in'KMGTPEZY']):
    pow,n=min(int(math.log(max(n*b**pow,1),b)),len(pre)-1),n*b**pow
    return "%%.%if %%s%%s"%abs(pow%(-pow-1))%(n/b**float(pow),pre[pow],u)

logging.basicConfig(filename=f'{folder}run.log',format='%(asctime)s - %(message)s', filemode='a+', level=logging.INFO)
def logfn(message, tq=True):
    logging.info(message)
    if tq:
        tqdm.write(message)
    else:
        print(message)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
for j in range(torch.cuda.device_count()):
    logfn(f"{j}: {pretty_size(torch.cuda.get_device_properties(j).total_memory)} of {'cuda' if torch.cuda.is_available() else 'cpu'} memory")

0: 48.0 GiB of cuda memory
1: 48.0 GiB of cuda memory


In [6]:
k = 100
ddp = False

dataset = graphPINN.data.MHSDataset(f'/mnt/d/data_k={k}',k=k)

propdesign = [12,6,3]
# propdesign = [12,3]
convdesign = [18,9,6,3]
# convdesign = [18,3]

propkernel = graphPINN.KernelNN(propdesign, torch.nn.ReLU)
propgraph = graphPINN.BDPropGraph(propkernel)
convkernel = graphPINN.KernelNN(convdesign, torch.nn.ReLU)
convgraph = graphPINN.ConvGraph(convkernel)

model = graphPINN.FullModel(propgraph, convgraph).to(device)

#trainset, validset, testset = torch.utils.data.random_split(dataset,[0.8, 0.1, 0.1],generator=torch.Generator().manual_seed(314))
trainset, validset, testset = torch.utils.data.random_split(dataset,[0.005, 0.001, 0.994],generator=torch.Generator().manual_seed(314))

In [7]:
epochs = 5
key = 0
lossdict = {'index_array':[[0,1,2],[3,4,5],[0,1,2],[3,4,5],-1]}

for index in tqdm(lossdict['index_array']):
    print(f'key {key} - index {index}')
    
    training_loss, validation_loss, state_dict = graphPINN.learn.train(
                model, trainset, validset, epochs=epochs,
                logfn=logfn, checkpointfile = f'{folder}_trainsize-{len(trainset)}_k-{k}')
    torch.save(state_dict, f'{folder}model_key-{key}_trainsize-{len(trainset)}_k-{k}.pt')
    
    lossdict[f'train{key}'] = training_loss.numpy()
    lossdict[f'valid{key}'] = validation_loss.numpy()
    logfn(f'training loss:\n{lossdict[f"train{key}"]}')
    logfn(f'validation loss:\n{lossdict[f"valid{key}"]}')
    savemat(f'{folder}loss_key-{key}_params-{math.prod(convdesign)+math.prod(propdesign)}.mat', lossdict)
    
    key = key + 1
    
lossdict['index_array'] = str(lossdict['index_array'])
torch.save(model, f'{folder}model_trainsize-{len(trainset)}_k-{k}_params-{math.prod(convdesign)+math.prod(propdesign)}.pt')
savemat(f'{folder}loss_{epochs}_trainsize-{len(trainset)}_k-{k}_params-{math.prod(convdesign)+math.prod(propdesign)}.mat', lossdict)

  0%|          | 0/5 [00:00<?, ?it/s]

key 0 - index [0, 1, 2]


  0%|          | 0/18 [00:00<?, ?it/s]

  iter 1/18, loss 5.718084812164307
  iter 2/18, loss 4.106081962585449
  iter 3/18, loss 10.801087379455566
  iter 4/18, loss 7.518902778625488


KeyboardInterrupt: 

In [ ]:
print(f'{pretty_size(torch.cuda.memory_allocated())}/{pretty_size(torch.cuda.max_memory_allocated())} allocated, ' +
      f'{pretty_size(torch.cuda.memory_reserved())}/{pretty_size(torch.cuda.get_device_properties(0).total_memory)} reserved')

In [ ]:
# convgraph.load_state_dict(torch.load('epoch-0_trainsize-3092_k-100.pt'))

In [ ]:
sample = dataset[0]
sample['bd','propagates','in'].edge_index, sample['bd','propagates','in'].edge_attr = \
            pyg.utils.dense_to_sparse(
                        torch.ones(sample['bd'].x.shape[0],sample['in'].x.shape[0])
            )
print(sample)

In [ ]:
homosamp = sample.to_homogeneous()

In [ ]:
print(sample['in'].x)

In [ ]:
print(homosamp.edge_index[:,homosamp.edge_type==1])

In [ ]:
lossdict

In [ ]:
mat = loadmat(f'{folder}loss_{epochs}_trainsize-{len(trainset)}_k-{k}_params-90.mat')

In [ ]:
for key in mat:
    print(key)
    print(mat[key])

In [ ]:
range(5)

In [ ]:
dataset = graphPINN.data.SHARPData(graphPINN.data._allsharps)

In [ ]:
sim = dataset.__getitem__(0)[0]
x_in = torch.zeros(sim[1].shape)
x_bd = sim[3]
y_in = sim[1]
y_bd = sim[3]
pos_in = sim[0]
pos_bd = torch.cat((sim[2],torch.zeros(sim[2].shape[0],1)),1)

p_in = sim[4]
p_bd = sim[5]

data = pyg.data.HeteroData()
data['in'].x = torch.cat((x_in,p_in),1)
data['in'].y = y_in
data['in'].pos = pos_in
data['in','adj','in'].edge_index = pyg.transforms.KNNGraph(k=50)(data['in']).edge_index
data['in'].edge_index = None

In [ ]:
trainLoader = pyg.loader.DataListLoader(trainset, batch_size=2,shuffle=False)
for data in trainLoader:
    for j in range(len(data)):
        data[j] = data[j].to_homogeneous()
    true = [torch.cat([datum.y[:,0:3] for datum in data]),
            torch.cat([datum.x[:,3] for datum in data]),
            torch.cat([datum.x[:,4] for datum in data]),
            torch.cat([datum.x[:,5] for datum in data])]

    pred = model(data)

    print(pred)
    print(true)
    break

In [ ]:
len('')